In [ ]:
import tushare as ts
from dotenv import load_dotenv
import os
Tushare_API_Key=os.getenv("Tushare_API_Key")

In [ ]:
import sqlite3
import pandas as pd
import tushare as ts
import time
from dotenv import load_dotenv
import os
# ==============================
# 1. Config
# ==============================
TUSHARE_TOKEN = os.getenv("Tushare_API_Key")
ts.set_token(TUSHARE_TOKEN)
pro = ts.pro_api()

DB_PATH = "stocks.db"
SYMBOLS=pd.read_csv("target_instruments/csi300.csv")['con_code'].unique().tolist()[200:300]

START_DATE = "20140101"
END_DATE = "20250930"

# ==============================
# 2. SQLite Init
# ==============================
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_data (
    _id TEXT PRIMARY KEY,           -- ts_code + trade_date
    symbol TEXT,
    date TEXT,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume REAL,
    adj_factor REAL
)
""")
conn.commit()

# ==============================
# 3. Fetch & Store
# ==============================
for symbol in SYMBOLS:
    print(f"Fetching: {symbol}")
    time.sleep(1)  # to respect API rate limits
    # --- daily price ---
    daily_df = pro.daily(
        ts_code=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        fields="ts_code,trade_date,open,high,low,close,vol"
    )

    # --- adj factor ---
    adj_df = pro.adj_factor(
        ts_code=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        fields="ts_code,trade_date,adj_factor"
    )

    # --- merge on ts_code + trade_date ---
    merged = pd.merge(
        daily_df,
        adj_df,
        on=["ts_code", "trade_date"],
        how="left"
    ).drop_duplicates(subset=["ts_code", "trade_date"])

    # --- rename columns to match table ---
    merged.rename(columns={
        "ts_code": "symbol",
        "trade_date": "date",
        "vol": "volume"
    }, inplace=True)

    # --- generate _id for uniqueness ---
    merged["_id"] = merged["symbol"] + "_" + merged["date"]

    # --- insert into SQLite ---
    try:
        merged.to_sql(
            "stock_data",
            conn,
            if_exists="append",
            index=False,
            method="multi"
        )
    except Exception as e:
        if e == "UNIQUE constraint failed: stock_data._id":
            continue
        else:
            print(f"Error occurred for {symbol}: {e}")
conn.commit()
conn.close()
print("✅ Data fetch and store complete.")


In [1]:
import sqlite3
import pandas as pd

df=pd.read_sql("SELECT * FROM stock_data LIMIT 5", sqlite3.connect("stocks.db"))
df

,_id,symbol,date,open,high,low,close,volume,adj_factor
0,600519.SH_20221230,600519.SH,20221230,1736.00,1752.99,1727.00,1727.00,25333.12,7.6508
1,600519.SH_20221229,600519.SH,20221229,1717.00,1726.99,1701.05,1719.00,22417.80,7.6508
2,600519.SH_20221228,600519.SH,20221228,1745.88,1747.00,1708.01,1733.00,21437.90,7.6508
3,600519.SH_20221227,600519.SH,20221227,1738.00,1747.15,1725.50,1733.00,17905.05,7.6508
4,600519.SH_20221226,600519.SH,20221226,1771.00,1771.00,1735.02,1742.06,21383.80,7.5546
